In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
response = requests.get(f"https://tasks.aidevs.pl/data/people.json")
people_list = json.loads(response.content)

print(people_list[0], end="\n\n")
print(len(people_list))

{'imie': 'Dariusz', 'nazwisko': 'Kaczor', 'wiek': 46, 'o_mnie': 'niekiedy lubie jeść lody. Mieszkam w Radomiu. Interesuję mnie polikyka a także żeglarstwo', 'ulubiona_postac_z_kapitana_bomby': 'Admirał Gwiezdnej Floty', 'ulubiony_serial': 'Stranger Things', 'ulubiony_film': 'Avengers', 'ulubiony_kolor': 'morski'}

1387


In [5]:
def get_ulubiona_postac_z_kapitana_bomby_of_person(name: str, last_name: str) -> str:
    found_people = [person for person in people_list if person["imie"] == name and person["nazwisko"] == last_name]
    person = found_people[0]
    return person["ulubiona_postac_z_kapitana_bomby"]

def get_ulubiony_serial_of_person(name: str, last_name: str) -> str:
    found_people = [person for person in people_list if person["imie"] == name and person["nazwisko"] == last_name]
    person = found_people[0]
    return person["ulubiony_serial"]

def get_ulubiony_film_of_person(name: str, last_name: str) -> str:
    found_people = [person for person in people_list if person["imie"] == name and person["nazwisko"] == last_name]
    person = found_people[0]
    return person["ulubiony_film"]

def get_ulubiony_kolor_of_person(name: str, last_name: str) -> str:
    found_people = [person for person in people_list if person["imie"] == name and person["nazwisko"] == last_name]
    person = found_people[0]
    return person["ulubiony_kolor"]

functions = {
    "get_ulubiona_postac_z_kapitana_bomby_of_person": get_ulubiona_postac_z_kapitana_bomby_of_person,
    "get_ulubiony_serial_of_person": get_ulubiony_serial_of_person,
    "get_ulubiony_film_of_person": get_ulubiony_film_of_person,
    "get_ulubiony_kolor_of_person": get_ulubiony_kolor_of_person
}

In [6]:
func_kapt_bomb_def = {
    "type": "function",
    "function": {
        "name": "get_ulubiona_postac_z_kapitana_bomby_of_person",
        "description": "Returns favourite character from Kapitan Bomba series of a given person",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Person's first name",
                },
                "last_name": {
                    "type": "string",
                    "description": "Person's last name",
                },
            },
            "required": ["name, last_name"],
        },
    },
}

func_series_def = {
    "type": "function",
    "function": {
        "name": "get_ulubiony_serial_of_person",
        "description": "Returns favourite TV series of a given person",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Person's first name",
                },
                "last_name": {
                    "type": "string",
                    "description": "Person's last name",
                },
            },
            "required": ["name, last_name"],
        },
    },
}

func_film_def = {
    "type": "function",
    "function": {
        "name": "get_ulubiony_film_of_person",
        "description": "Returns favourite movie of a given person",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Person's first name",
                },
                "last_name": {
                    "type": "string",
                    "description": "Person's last name",
                },
            },
            "required": ["name, last_name"],
        },
    },
}

func_kapt_bomb_def = {
    "type": "function",
    "function": {
        "name": "get_ulubiony_kolor_of_person",
        "description": "Returns favourite color of a given person",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Person's first name",
                },
                "last_name": {
                    "type": "string",
                    "description": "Person's last name",
                },
            },
            "required": ["name, last_name"],
        },
    },
}

In [14]:
response = requests.post("https://tasks.aidevs.pl/token/people", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)
question = response_json["question"]
print(response_json)

{'code': 0, 'msg': 'retrieve the data set (JSON) and answer the question. The question will change every time the task is called. I only ask about favourite colour, favourite food and place of residence', 'data': 'https://tasks.aidevs.pl/data/people.json', 'question': 'jaki kolor się podoba Mariuszowi Kaczorowi?', 'hint1': 'Does everything have to be handled by the language model?', 'hint2': 'prepare knowledge DB for this task'}


In [15]:
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "user", "content": question}
  ],
  tools=[func_kapt_bomb_def, func_series_def, func_film_def, func_kapt_bomb_def],
  tool_choice="auto"
)

message = completion.choices[0].message
print(message, end="\n\n")
if message.tool_calls:
    func_name = message.tool_calls[0].function.name
    func_args = json.loads(message.tool_calls[0].function.arguments)
    print(func_name)
    print(func_args)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ulnBf7oachczUsuMSQd7lvHl', function=Function(arguments='{\n  "name": "Mariusz",\n  "last_name": "Kaczor"\n}', name='get_ulubiony_kolor_of_person'), type='function')])

get_ulubiony_kolor_of_person
{'name': 'Mariusz', 'last_name': 'Kaczor'}


In [16]:
answer = functions[func_name](**func_args)
print(answer)

koralowy


In [17]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=answer))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
